# Content Based Recommendation

In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from urllib.request import urlopen
import matplotlib.pyplot as plt
import math
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('movie_info1.csv')

In [3]:
df = df.dropna()


In [4]:
df.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Plot_reduction
0,1971,The Horsemen,American,John Frankenheimer,"Omar Sharif, Leigh Taylor-Young, Jack Palance",adventure,https://en.wikipedia.org/wiki/The_Horsemen_(19...,"Uraz (Omar Sharif), the son of Tursen (Jack Pa...",stabl retir journey feudal risk live seek prim...
1,1971,The Light at the Edge of the World,American,Kevin Billington,"Kirk Douglas, Yul Brynner, Samantha Eggar",adventure,https://en.wikipedia.org/wiki/The_Light_at_the...,The year is 1865. Will Denton (Kirk Douglas) i...,jade american escap troubl two mend broken fai...
2,1971,Sometimes a Great Notion,American,Paul Newman,"Paul Newman, Henry Fonda, Lee Remick, Michael ...",adventure,https://en.wikipedia.org/wiki/Sometimes_a_Grea...,"The economic stability of Waconda, Oregon, is ...",econom threaten local call larg independ urg j...
3,1972,Napoleon and Samantha,American,Bernard McEveety,"Michael Douglas, Jodie Foster",adventure,https://en.wikipedia.org/wiki/Napoleon_and_Sam...,Eleven-year-old Napoleon lives with his grandf...,eleven old live adopt name meet old take back ...
4,1972,The Poseidon Adventure,American,Ronald Neame,"Gene Hackman, Ernest Borgnine, Shelley Winters...","disaster film, adventure",https://en.wikipedia.org/wiki/The_Poseidon_Adv...,"The S.S. Poseidon, an ocean liner slated for r...",ocean slate despit fear new repres go full sav...




## Wiki Plot Preprocessing



In [5]:
def PlotPreprocessing(pl):
    i = pl.find('==Plot==')
    if(i!=-1):
        pl = pl[len('==plot=='):]
    while(1):
        i = pl.find('[[')
        if(i==-1):
            break
        j = pl.find(']]')
        if(i>j):
            pl = pl[:j] + pl[j+1:]
            j = pl.find(']]')
        pl = pl[:i] + pl[j+2:]
    while(1):
        i = pl.find('{{')
        if(i==-1):
            break
        j = pl.find('}}')
        if(i>j):
            pl = pl[:j] + pl[j+1:]
            j = pl.find('}}')
        pl = pl[:i] + pl[j+2:]
    while(1):
        i = pl.find('(')
        if(i==-1):
            break
        j = pl.find(')')
        if(i>j):
            pl = pl[:j] + pl[j+1:]
            j = pl.find(')')
        pl = pl[:i] + pl[j+2:]
    while(1):
        i = pl.find('<')
        if(i==-1):
            break
        j = pl.find('>')
        if(i>j):
            pl = pl[:j] + pl[j+1:]
            j = pl.find('>')
        pl = pl[:i] + pl[j+2:]
    p = ''
    for i in pl:
        if(i!='\n'):
            p += i
    return p
        

In [16]:
movie_name = input('Enter the movie name : ')
movie_name = movie_name.title()
movie = ""
for i in range(len(movie_name)):
    if movie_name[i] == ' ':
        movie += '+'
    else:
        movie += movie_name[i]
# print(movie)
url = 'https://www.google.com/search?q='+movie
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}
response = requests.get(url,headers = headers)
soup = BeautifulSoup(response.text, 'lxml')


#print(soup)
gen = soup.find_all('div',{'class':'BNeawe'})[1].text.split()[2].lower()
print('genre : ',gen)
fl = 0
cast = ''
for g in soup.find_all('div',{'class':'BNeawe'}):
    if fl == 1:
        classes = g['class']
        if 's3v9rd' not in classes and 'tAd8D' not in classes:
            fl = 0
        k+=1
        if(k == 8):
            fl = 0
        if 's3v9rd' in classes:
            if cast == '':
                cast = g.text
            else:
                cast = cast + ',' + g.text
    ind = g.text.find('Director: ')
    if  ind != -1:
        director = g.text[ind + len('Director:'):].strip()
    if g.text == 'Cast':
        fl = 1
        k = 0
ind = director.find('\n')
if ind != -1:
    director = director[:ind]
ind = director.find('.')
if ind != -1:
    director = director[:ind]
print('Director =',director)
print('Cast = ',cast)
for g in soup.find_all('div',{'class':'kCrYT'}):
    if(g.find('a') != None):
        if('wiki' in g.find('a').get('href')):
            url = g.find('a').get('href')
            ind = url.find('url=')
            url = url[ind+4:]
            ind = url.find('&')
            url = url[:ind]
            break

print('movie url : ',url)
html = urlopen(url) 
soup = BeautifulSoup(html, 'html.parser')
url = "https://en.wikipedia.org"+soup.find('a',{'title':'Edit section: Plot'})['href']
print('edit section url : ',url)
html = urlopen(url) 
soup = BeautifulSoup(html, 'html.parser')
plot = soup.find('textarea').text
print('Plot : ',PlotPreprocessing(plot))

Enter the movie name : Mersal
genre :  action/drama
Director = Atlee Kumar
Cast =  Vijay,Nithya Menen,Samantha Ruth Prabhu,Kajal Aggarwal
movie url :  https://en.wikipedia.org/wiki/Mersal_(film)
edit section url :  https://en.wikipedia.org/w/index.php?title=Mersal_(film)&action=edit&section=1
Plot :  == Plot ==An ambulance driver, a broker, a hospital worker, and a surgeon are abducted at dawn, which is traced to Dr. Maaran, a doctor from  known for providing treatment to all at just . Maaran is arrested and interrogated by Inspector Ratnavel "Randy", who is assigned the case. Maaran explains his motive for the abductions; the four were responsible for the death of an auto driver's daughter and subsequent suicide of his wife due to their greed for money and negligence in providing proper healthcare. He gives Randy the locations of his hostages but asserts that he had already killed them simultaneously.Maaran reveals that he is not Maaran but his  Vetri, a . Vetri was also responsible f

## Plot Extraction from Wiki

In [6]:
def ExtractPlot(df):
    for i in df.index:
        url = df.loc[i]['Wiki Page']
        html = urlopen(url) 
        soup = BeautifulSoup(html, 'html.parser')
        try:
            url = "https://en.wikipedia.org"+soup.find('a',{'title':'Edit section: Plot'})['href']
        except:
            continue
        html = urlopen(url) 
        soup = BeautifulSoup(html, 'html.parser')
        plot = soup.find('textarea').text
        try:
            i = plot.index('==')
            plot = plot[i+len('=='):]
            i = plot.index('==')
            plot = plot[i+len('=='):]
        except:
            plot = plot
            df.loc[i,'Plot'] = 'a'
        plot = func(plot)
        df.loc[i,'Plot'] = plot

In [7]:
import nltk
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
ps = PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SERJIUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\SERJIUS\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SERJIUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Word Stemming and Stop Words Removal

In [8]:
def SWRandStem(plot):
    p = ''

    for i in plot:
        if(i!='\n'):
            p += i
    plot = p
    stop_words = set(stopwords.words('english'))
    tagged_sentence = nltk.tag.pos_tag(plot.split())
    edited_sentence = [word for word,tag in tagged_sentence if tag != 'NNP' and tag != 'NNPS']
    example_sentence = ' '.join(edited_sentence)
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~0123456789'''
    no_punct = ""
    for char in example_sentence:
       if char not in punctuations:
           no_punct = no_punct + char
       else:
            no_punct += ' '
    example_sentence = no_punct
    tagged_sentence = nltk.tag.pos_tag(example_sentence.split())
    edited_sentence = [word for word,tag in tagged_sentence if tag not in ['NNP','NNPS','NNS','NN']]
    example_sentence = ' '.join(edited_sentence)
    example_sentence = example_sentence.lower()
    stop_words = set(stopwords.words('english')) 

    word_tokens = word_tokenize(example_sentence) 

    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    filt = [ps.stem(filtered_sentence[i]) for i in range(len(filtered_sentence))]
    filt = ' '.join(filt)
    return filt

In [ ]:
from csv import writer
sent=SWRandStem(plot)
data=[movie_name,gen,sent]
with open('input_plot.csv', 'a') as f_object:

    writer_object = writer(f_object)

    writer_object.writerow(data)

    f_object.close()

# Selecting the Best Topic

In [9]:
gen = ['sci_fi','fantasy','crime','adventure','music']
cons = pd.DataFrame()
def SelectTopic(genre,gen_name):
    global cons
    genre = genre[genre['Plot_reduction']!= np.nan]
    th_plot = genre['Plot_reduction']
    th_plot = th_plot.dropna()

    th_plot_80,th_plot_20 = train_test_split(th_plot,train_size = int(len(th_plot)*0.8),\
                                             test_size = len(th_plot)-int(len(th_plot)*0.8))

    if(gen_name in gen):
        vectorizer = TfidfVectorizer(min_df = 50,stop_words='english',max_df = 0.9,max_features = 500)
    else:
        vectorizer = TfidfVectorizer(min_df = 200,stop_words='english',max_df = 0.9,max_features = 500)
    th_plot = th_plot_80.append(pd.Series(th_plot_20))

    bagofwords = vectorizer.fit_transform(th_plot)
    feature_names = vectorizer.get_feature_names()
    dense = bagofwords.todense()
    denselist = dense.tolist()
    df = pd.DataFrame(denselist, columns=feature_names)
    rat = df.iloc[df.shape[0]-len(th_plot_20):]

    ncomp = 15
    svd = TruncatedSVD(n_components = ncomp)
    lsa = svd.fit_transform(bagofwords)
    
    diction30 = vectorizer.get_feature_names()

    topic = ['Topic '+str(i) for i in range(1,ncomp+1)]
    
    encoding_matrix = pd.DataFrame(svd.components_,index = topic).T
    encoding_matrix['words'] = diction30
    diction30 = encoding_matrix.sort_values(by=['Topic 1'],ascending = False)
    diction30.index = diction30['words']
    diction30 = diction30.drop(['words'],axis = 1)
#     cons = diction30
    rat = rat.T
    rat.index.set_names(['words'],inplace = True)
    comp = pd.concat([rat,diction30], axis=1, join='inner').T
#     cons = comp
    c = cosine_similarity(comp,comp)
    final = pd.DataFrame(c[len(c[0])-ncomp:,:comp.shape[0]-ncomp],index = topic)
    
    
    final['sum'] = final.sum(axis = 1,skipna = True)
    cons = final['sum']
    return final.idxmax(axis = 0,skipna = True)['sum']

## Genre Similarity

In [10]:
gen = ['sci_fi','fantasy','crime','adventure','music']
ncomp =15
def FindSimilarity(genre,summa,gen_name,title):
    global cons
    global cons1
    genre = genre[genre['Plot_reduction']!= np.nan]
    th_plot = genre['Plot_reduction']
    th_plot = th_plot.append(pd.Series(summa))

    if(gen_name in gen):
        vectorizer = TfidfVectorizer(min_df = 50,stop_words='english',max_df = 0.9,max_features = 500)
    else:
        vectorizer = TfidfVectorizer(min_df = 200,stop_words='english',max_df = 0.9,max_features = 500)
    bagofwords = vectorizer.fit_transform(th_plot)
    feature_names = vectorizer.get_feature_names()
    dense = bagofwords.todense()
    denselist = dense.tolist()
    df2 = pd.DataFrame(denselist, columns=feature_names)
    cons1 = df2
    rat = df2.iloc[df2.shape[0]-len(summa):]
    rat = rat.T
    rat.index.set_names(['words'],inplace = True)

    svd = TruncatedSVD(n_components = ncomp)
    lsa = svd.fit_transform(bagofwords)
    diction30 = vectorizer.get_feature_names()
    
    top = SelectTopic(genre,gen_name)
    topic = ['Topic '+str(i) for i in range(1,ncomp+1)]
    cons = feature_names

    encoding_matrix = pd.DataFrame(svd.components_,index = topic).T
    cons = encoding_matrix
    encoding_matrix['words'] = diction30
    diction30 = encoding_matrix.sort_values(by=['Topic 1'],ascending = False)
    diction30.index = diction30['words']
    topic.remove(top)
    diction30 = diction30.drop(topic,axis = 1)
    diction30 = diction30.drop(['words'],axis = 1)

    comp = pd.concat([rat,diction30], axis=1, join='inner').T
    comp = comp.drop(['tell','ask','come','want','make','say','young'], axis=1, errors='ignore')

    if gen_name not in ['horror','thriller','action']:
        comp = comp.drop(['kill'],axis = 1,errors = 'ignore')

    c = cosine_similarity(comp,comp)

    temp = pd.DataFrame(c[len(c[0])-1],index = title)
    temp.rename(columns = {0:gen_name},inplace = True)
    return temp.T

In [11]:
action = df[df.apply(lambda x: "action" in x['Genre'].lower(),axis = 1)]
drama = df[df.apply(lambda x: "drama" in x['Genre'].lower() , axis=1)]
fantasy = df[df.apply(lambda x: "fantasy" in x['Genre'].lower(),axis = 1)]
horror = df[df.apply(lambda x: "horror" in x['Genre'].lower(),axis = 1)]
sci_fi = df[df.apply(lambda x: "sci" in x['Genre'].lower(),axis = 1)]
romance = df[df.apply(lambda x: "romance" in x['Genre'].lower() or "romantic"  in x['Genre'].lower(),axis = 1)]
thriller = df[df.apply(lambda x: "thriller" in x['Genre'] or "mystery" in x['Genre'].lower(),axis = 1)]
crime = df[df.apply(lambda x: "crime" in x['Genre'].lower(),axis = 1)]
adventure = df[df.apply(lambda x: "adventure" in x['Genre'].lower(),axis = 1)]

## Prediction of the Test Dataset

In [12]:
first_h = 10
genre_df = [action,adventure,crime,drama,fantasy,horror,romance,sci_fi,thriller]
genre_nm = ['action','adventure','crime','drama','fantasy','horror','romance','sci_fi','thriller']
def PredictSimilarity(plot,title):
    df = pd.DataFrame()
    df1 = pd.DataFrame()
    for j in range(len(plot)):
        for i in range(len(genre_df)):
            a = FindSimilarity(genre_df[i],[plot[j]],genre_nm[i],[title[j],'temp'])
            df = pd.concat([df,a])
            
            c = cosine_similarity(cons1,cons1)
            temp = pd.DataFrame(c[len(genre_df[i]):,:len(genre_df[i])],index = [title[j]])
            temp.columns = genre_df[i]['Title']
            for i in temp.index:
                if type(fin[j]) != pd.Series:
                    fin[j] = temp.loc[i].sort_values(ascending = False)[:first_h]
                else:
                    fin[j] = fin[j].append(temp.loc[i].sort_values(ascending = False)[:first_h])
                    fin[j] = fin[j].drop_duplicates()
                    fin[j] = fin[j].sort_values(ascending = False)[:first_h]
        print(j,end = ' ')
        

        df = df.drop(['temp'],axis = 1,errors = 'ignore')
        df1 = pd.concat([df1,df],axis = 1)
        df = pd.DataFrame()
    
    return df1    

In [13]:
def softmax(x):
    e_x = np.exp(x)
    return x / x.sum()


## Recommendation Score 

In [17]:
def FindAccuracy(ans):
    cumu_sum = 0
    for ind,col in enumerate(ans.columns):
        wt = [ans[col][i] * 100 for i in range(len(ans[col]))]
        sumi = 0

        for i in range(first_h):
            sumj = 0
            for j in range(len(genre_df)):
                a1=FindSimilarity(genre_df[j],\
                      df[df['Title'] == fin[ind].index[i]][:1]['Plot_reduction'].values,\
                      genre_nm[j],[fin[ind].index[i],'Topic1']).iloc[0,0]
                a2,a3 = ans[col][genre_nm[j]],fin[ind].values[i]
                a1,a2,a3 = softmax(np.array([a1,a2,a3]))
                mx = max(a1,a2,a3)
                sumj += (a1/mx + a2/mx + a3/mx)/3*wt[j]
            sumi += (sumj/sum(wt))
        cumu_sum += (sumi*100)
        print(ind,end = ' ')
    format_c = '{:.2f}'.format(cumu_sum/(len(ans.columns))/10)
    print('\nRecomendation Score : '+format_c+'/100')
    return cumu_sum/10/(len(ans.columns))


In [18]:
inp = pd.read_csv('input_plot.csv')
fin = [None for i in range(len(inp))]
ans = PredictSimilarity(inp['Plot_reduction'],inp['title'])

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 

In [21]:
fin[0]

Title
Pawn                                    0.778800
Shocker                                 0.777774
Deadfall                                0.766957
Ee Adutha Kalathu (ഈ അടുത്ത കാലത്ത്)    0.764180
Australia                               0.759452
Freelance                               0.740670
Tattoo                                  0.740149
Naku Penta Naku Taka                    0.739304
Hammett                                 0.733938
Man-Thing                               0.723099
Name: ratchasan, dtype: float64

In [22]:
ans

,ratchasan,96,santhosh subramaniyam,kutram 23,vtv,thozha,silence,anandham,zodiac,the silence of the lambs,...,Viswasam,Harry Potter and the Philosopher's Stone,Maya,The Conjuring,Now You See Me,Good Fellas,Enthiran,Shutter Island,Soorarai Potru,Karnan
action,0.458806,0.247055,0.274224,0.437196,0.256206,0.276975,0.528012,0.280074,0.405475,0.487408,...,0.550614,0.405831,0.351979,0.371558,0.348680,0.363638,0.313419,0.397443,0.304383,0.387448
adventure,0.242600,0.190242,0.246525,0.302848,0.216153,0.277365,0.363746,0.238982,0.283763,0.316657,...,0.341311,0.361398,0.289157,0.338960,0.318799,0.323306,0.286558,0.290530,0.292169,0.313696
crime,0.302169,0.170987,0.217991,0.328189,0.236445,0.235912,0.357079,0.238227,0.267117,0.299495,...,0.270394,0.283523,0.289781,0.269764,0.494229,0.615708,0.208135,0.280557,0.295939,0.275752
drama,0.188556,0.235667,0.453189,0.247129,0.308780,0.571308,0.282007,0.485333,0.234671,0.240226,...,0.405210,0.243782,0.239550,0.210945,0.165934,0.266627,0.161125,0.230523,0.272719,0.308458
fantasy,0.451376,0.298064,0.350654,0.421919,0.376725,0.383334,0.486407,0.372243,0.411706,0.519554,...,0.464542,0.743536,0.392137,0.462961,0.592227,0.423248,0.412491,0.426401,0.475824,0.444232
horror,0.355500,0.171024,0.220470,0.400027,0.284929,0.250987,0.532003,0.271527,0.446992,0.519433,...,0.334007,0.441606,0.615502,0.722797,0.320260,0.385575,0.299767,0.410570,0.252913,0.320350
romance,0.263091,0.683040,0.676998,0.281375,0.687418,0.437195,0.433516,0.567790,0.300601,0.296649,...,0.523907,0.318640,0.351447,0.335481,0.209584,0.514914,0.509031,0.248391,0.368439,0.374336
sci_fi,0.290915,0.172670,0.234728,0.372594,0.303082,0.307951,0.414208,0.281295,0.335515,0.349897,...,0.359967,0.414276,0.358748,0.357777,0.321146,0.354255,0.611615,0.354377,0.333854,0.353024
thriller,0.745720,0.275500,0.358863,0.705330,0.383990,0.290501,0.721022,0.359951,0.664420,0.735625,...,0.429655,0.559172,0.487563,0.445529,0.325299,0.496706,0.342373,0.761094,0.343955,0.354524


In [22]:
ans.to_csv('output.csv')

PermissionError: [Errno 13] Permission denied: 'output.csv'

In [23]:
FindAccuracy(ans)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 
Recomendation Score : 70.50/100


70.49672305649882

# Collaborative Filtering 

In [23]:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('ml-100k/u.user', sep='|', names=u_cols,encoding='latin-1')

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=r_cols,encoding='latin-1')

i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical',
          'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

items = pd.read_csv('ml-100k/u.item', sep='|', names=i_cols,encoding='latin-1')
org_items = len(items)
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_train = pd.read_csv('ml-100k/ua.base', sep='\t', names=r_cols, encoding='latin-1')

ratings_test = pd.read_csv('ml-100k/ua.test', sep='\t', names=r_cols, encoding='latin-1')

ratings_train.shape, ratings_test.shape
ratings

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


## User-Item prediction

In [24]:

for i in items['movie id']:
    if(len(ratings[ratings['movie_id'] == i]) <= 10):
        items.drop(i-1,inplace = True)
        ratings.drop(ratings[ratings['movie_id'] == i].index,inplace = True)

def predict(ratings, similarity, type='user'):
    
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1).reshape(-1,1)
        
        ratings_diff = (ratings - mean_user_rating)
        pred = mean_user_rating + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    
    return pred

In [25]:
items = items.reset_index()

In [26]:
n_users = ratings.user_id.unique().shape[0]
n_items = ratings.movie_id.unique().shape[0]
print(n_users,n_items)

943 1119


In [27]:
data_matrix = np.zeros((n_users, n_items))
for line in ratings.itertuples():
    data_matrix[line[1]-1, items[items['movie id'] == line[2]].index.values[0] ] = line[3]

from sklearn.metrics.pairwise import pairwise_distances 
user_similarity = pairwise_distances(data_matrix, metric='cosine')
user_prediction = predict(data_matrix, user_similarity, type='user')
user_pred = pd.DataFrame(user_prediction)

item_similarity = pairwise_distances(data_matrix.T, metric='cosine')
item_prediction = predict(data_matrix, item_similarity, type='item')
item_pred = pd.DataFrame(item_prediction)




In [28]:
user_pred

,0,1,2,3,4,5,6,7,8,9,...,1109,1110,1111,1112,1113,1114,1115,1116,1117,1118
0,2.235147,0.905309,0.802637,1.179952,0.813373,0.649508,1.953717,1.332476,1.684132,0.876699,...,0.592456,0.600405,0.605281,0.603287,0.618797,0.600252,0.604867,0.587363,0.605689,0.595257
1,1.721155,0.341706,0.153312,0.689435,0.183163,-0.039437,1.451273,0.834104,1.065142,0.218675,...,-0.095335,-0.083947,-0.078534,-0.081460,-0.066320,-0.088495,-0.079027,-0.103359,-0.090631,-0.093384
2,1.733892,0.265929,0.095588,0.621250,0.110092,-0.099101,1.425780,0.772978,1.072625,0.173057,...,-0.162878,-0.153909,-0.148191,-0.149763,-0.135525,-0.158925,-0.148371,-0.173274,-0.161847,-0.162680
3,1.646599,0.208889,0.042018,0.560581,0.056682,-0.147971,1.352342,0.712612,1.012281,0.126791,...,-0.211812,-0.201886,-0.198297,-0.199491,-0.184737,-0.208753,-0.197433,-0.222600,-0.209046,-0.211247
4,1.833881,0.493106,0.394702,0.800791,0.399657,0.236958,1.585258,0.946607,1.331683,0.478224,...,0.170942,0.178991,0.178287,0.178943,0.195748,0.172551,0.183013,0.160858,0.179643,0.172466
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,1.634921,0.304397,0.135463,0.648047,0.157835,-0.038829,1.387420,0.789011,1.028555,0.220079,...,-0.100817,-0.090443,-0.086544,-0.089210,-0.071395,-0.098126,-0.086420,-0.110210,-0.093377,-0.100714
939,1.834629,0.430399,0.298027,0.726663,0.305992,0.117922,1.526185,0.863983,1.206493,0.370406,...,0.054573,0.065137,0.069030,0.068257,0.080032,0.062828,0.069129,0.048257,0.062057,0.056653
940,1.512465,0.196274,0.023087,0.545502,0.054860,-0.148657,1.241485,0.686661,0.955857,0.113594,...,-0.210165,-0.201302,-0.196955,-0.198497,-0.182763,-0.207030,-0.195755,-0.221167,-0.204883,-0.209526
941,1.811383,0.405065,0.276039,0.726800,0.281799,0.087434,1.551089,0.850030,1.206129,0.343305,...,0.021760,0.033368,0.033057,0.036058,0.042393,0.028189,0.036876,0.014899,0.028845,0.023801


In [29]:
item_pred

,0,1,2,3,4,5,6,7,8,9,...,1109,1110,1111,1112,1113,1114,1115,1116,1117,1118
0,0.695108,0.732261,0.768520,0.691876,0.776530,0.804270,0.693761,0.715756,0.729759,0.776204,...,0.827081,0.822338,0.826941,0.826365,0.830749,0.847956,0.821433,0.843702,0.840898,0.832929
1,0.173203,0.209972,0.194437,0.200298,0.202680,0.192151,0.176658,0.193793,0.171772,0.186274,...,0.203740,0.205072,0.208352,0.205157,0.205723,0.210255,0.206262,0.211054,0.197508,0.205234
2,0.140326,0.147867,0.138660,0.147642,0.141411,0.136493,0.136492,0.145628,0.136947,0.138308,...,0.142039,0.137547,0.143169,0.141055,0.140575,0.141523,0.139378,0.142590,0.128935,0.139171
3,0.076324,0.084774,0.081256,0.083995,0.082965,0.083616,0.075500,0.083675,0.078705,0.084403,...,0.086742,0.085210,0.086677,0.085318,0.086304,0.085047,0.084627,0.087019,0.080592,0.085349
4,0.349602,0.352207,0.399179,0.352867,0.393322,0.433752,0.362238,0.366299,0.397460,0.413498,...,0.427081,0.422173,0.403575,0.412337,0.426602,0.415487,0.417341,0.413990,0.425756,0.425065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,0.151608,0.183636,0.174363,0.183868,0.177861,0.186784,0.161154,0.179896,0.158921,0.185671,...,0.190875,0.190198,0.190900,0.185610,0.194984,0.185581,0.188283,0.192501,0.186315,0.185153
939,0.269168,0.298175,0.310883,0.270590,0.308917,0.318010,0.266209,0.268786,0.276119,0.304603,...,0.320448,0.324534,0.323808,0.326377,0.318625,0.335160,0.322685,0.332467,0.321691,0.322650
940,0.052898,0.072610,0.067910,0.070933,0.074907,0.077477,0.053558,0.069217,0.063336,0.074252,...,0.081450,0.078837,0.080494,0.079843,0.081172,0.080677,0.079919,0.081057,0.078125,0.080482
941,0.258395,0.280760,0.299015,0.268342,0.293586,0.294498,0.269593,0.254769,0.268526,0.283870,...,0.291866,0.299990,0.287385,0.298291,0.280405,0.303767,0.297280,0.303241,0.292097,0.293820


## User-User Recommendation

In [30]:
first_h = 10
cols = items.columns[6:]

def UserUserRecommendation(userid):
    user_indv = ratings[ratings['user_id'] == userid]['movie_id'].values
    user_pred_indv = items['movie id'][user_pred.iloc[userid-1].sort_values(ascending = False).index.values]
    diff = np.setdiff1d(user_pred_indv, user_indv)
    diff = [items[items['movie id'] == diff[i]].index.values[0] for i in range(len(diff))]
    rec_mov = user_pred.iloc[0,diff].sort_values(ascending = False)[:first_h]
    li = [ list(cols[(items.iloc[rec_mov.index.values,6:].values == 1)[i]].values) for i in range(first_h)]
    li = [','.join(i) for i in li]
    return [i+' -- '+j for i,j in zip(list(items.iloc[rec_mov.index.values]['movie title']),li)]



In [31]:
UserUserRecommendation(6)

['Return of the Jedi (1983) -- Action,Adventure,Romance,Sci-Fi,War',
 'Scream (1996) -- Horror,Thriller',
 'Air Force One (1997) -- Action,Thriller',
 'Titanic (1997) -- Action,Drama,Romance',
 'Independence Day (ID4) (1996) -- Action,Sci-Fi,War',
 'Empire Strikes Back, The (1980) -- Action,Adventure,Drama,Romance,Sci-Fi,War',
 'Star Trek: First Contact (1996) -- Action,Adventure,Sci-Fi',
 'Indiana Jones and the Last Crusade (1989) -- Action,Adventure',
 'Conspiracy Theory (1997) -- Action,Mystery,Romance,Thriller',
 'Saint, The (1997) -- Action,Romance,Thriller']

In [32]:
item_df = pd.DataFrame(item_similarity)

## Item-Item Recommendation

In [33]:
def ItemItemRecommendation(userid):
    pred = None
    for i in ratings[ratings['user_id'] == userid].sort_values(by = ['unix_timestamp'])['movie_id']:
        ind = items[items['movie id'] == i].index.values[0]
        k = item_df[ind]
        k = k.sort_values(ascending = False)
        k = k[:10]
        if type(pred) != pd.Series:
            pred = k
        else:
            pred.append(k)
            pred = pred.sort_values(ascending = False)[:10]

    rec_mov = pred
    lis = [ list(cols[(items.iloc[rec_mov.index.values,6:].values == 1)[i]].values) for i in range(first_h)]
    lis = [','.join(i) for i in lis]
    return [i+' -- '+j for i,j in zip(list(items.iloc[rec_mov.index.values]['movie title']),lis)]

In [34]:
ItemItemRecommendation(943)

['Truman Show, The (1998) -- Drama',
 "Gone Fishin' (1997) -- Comedy",
 'Half Baked (1998) -- Comedy',
 'Telling Lies in America (1997) -- Drama',
 'Four Days in September (1997) -- Drama',
 'Palmetto (1998) -- Film-Noir,Mystery,Thriller',
 'One Night Stand (1997) -- Drama',
 'Switchback (1997) -- Thriller',
 'Oscar & Lucinda (1997) -- Drama,Romance',
 'Welcome To Sarajevo (1997) -- Drama,War']

In [35]:
genre_colab = ['Action', 'Adventure','Animation', 'Children\'s', 'Comedy', 'Crime', 
               'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 
               'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
def UserPreference(userid):
    genre_score = [0]*len(genre_colab)
    for i in ratings[ratings['user_id'] == userid]['movie_id']:
        lis =  list(cols[(items[items['movie id']==i].iloc[:,6:].values == 1)[0]].values)
        for j in range(len(lis)):
            genre_score[genre_colab.index(lis[j])] += 1
    return genre_score

In [36]:
def UserScore(userid,typeRecom):
    new_genre_score = [0]*len(genre_colab)
    gen_score = UserPreference(userid)
    if(typeRecom == 'user'):
        pred = UserUserRecommendation(userid)
    else:
        pred = ItemItemRecommendation(userid)
    for i in range(first_h):
        rec = pred[i].split('--')
        rec = rec[-1]
        rec = rec.strip()
        rec = rec.split(',')
        for i in rec:
            try:
                new_genre_score[genre_colab.index(i)] += 1
            except:
                new_genre_score[gen_score.index(max(gen_score))]
    s = sum(new_genre_score)
    new_genre_score = [i/s*100 for i in new_genre_score]
    
    return (np.array(gen_score)*np.array(new_genre_score)/np.array(new_genre_score).sum()).sum()


## Recommendation Score

In [37]:
def ReccomScore(typeRecom):
    count = 0
    sumre = 0
    for i in range(1,len(users)+1):
        if(len(ratings[ratings['user_id'] == i])>250):
            sumre += UserScore(i,typeRecom)
            count+=1
    return sumre/count

In [38]:
format_c = '{:.2f}'.format(ReccomScore('user'))
print('Recommendation Score for user-user recommendation: '+format_c+'/100')

Recommendation Score for user-user recommendation: 67.29/100


In [39]:
format_c = '{:.2f}'.format(ReccomScore('item'))
print('Recommendation Score for item-item recommendation: '+format_c+'/100')

Recommendation Score for item-item recommendation: 66.45/100
